# 1. Entendimento do Problema de Negócio

Indaial é uma cidade de Santa Catarina com alto IDH (0.777). Ela tem cerca de <b>70 mil</b> habitantes e conta com <b>13</b> vereadores em sua câmara municipal.

![](imagens/SantaCatarina_Municip_Indaial.png)

Neste projeto vamos analisar as <b>proposições</b> escritas pelos vereadores de Indaial desde <b>1996 até 2023</b>. Ou seja, vamos explorar os projetos de lei, projetos de resolução, requerimentos, indicações ou moções submetidas pelos vereadores para tramitação na Câmara e discussão no plenário.

Isto é importante porque conseguiremos avaliar a participação política dos vereadores eleitos, quais são os temas mais discutidos na câmara, o tempo médio entre uma reunião e uma deliberação, qual a situação geral das proposições e qual é o tema/texto que teve mais aprovação ao longo dos anos.

Este tipo de projeto pode auxiliar aos eleitores a ter um melhor entendimento da gestão pública e do funcionamento da democracia na cidade de Indaial. Além disso, pode facilitar aos meios de comunicação uma busca mais automatizada da atuação dos vereadores.

Para fazer tudo isso, vamos utilizar a linguagem <b>Python</b> para retirar os dados do site da prefeitura de forma automatizada, o <b>SQL</b> para inserir estes registros em um banco de dados e o <b>Power BI</b> para visualizar os insights que podem ser obtidos destes dados.

# 2. Webscrapping do site da Prefeitura de  Indaial

Vamos utilizar o site da prefeitura para conseguir obter os registros sobre as proposições de forma automatizada!

Inicialmente vamos setar algumas bibliotecas que serão utilizadas em nosso código e uma constante que representa o conjunto de informações enviadas por um navegador para um servidor web para informar o tipo do navegador, sistema operacional, etc.

In [1]:
# importando as bibliotecas
import pandas as pd # utilizado para trabalhar com tabelas
import bs4 # utilizado para acessar o código do site da cidade
import requests # fazer as requisições para acessar o código do site da cidade

# variáveis constantes utilizadas ao longo do código
user_agent = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
              "AppleWebKit/537.36 (KHTML, like Gecko)"
              "Chrome/110.0.0.0" 
              "Safari/537.36" 
              "Edg/110.0.1587.63")

Nosso objetivo inicial é acessar cada uma das proposições e obter o código HTML da página.

![](imagens/print_proposicao_162_2023.png)

In [2]:
# url de exemplo de uma proposição do ano de 2023
url = ("https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1"
             "&nrProposicao=162" # Número da proposição
             "&aaProposicao=2023") # Ano da proposição

# adicionando as constantes definidas no inicio do código
headers = {'User-Agent': user_agent}

# obtendo o conteúdo html da página
html = requests.get(url, headers = headers, timeout  = 10).content

# passando o html para o BeautifulSoup
soup = bs4.BeautifulSoup(html, 'html.parser')

# imprimindo o código html de forma visual
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <link href="legis.ico" rel="shortcut icon" type="image/x-ico"/>
  <title>
   Câmara Municipal de Indaial _ Indicação nº 162/2023 de 02/03/2023
  </title>
  <meta content="Câmara Municipal de Indaial _ Indicação nº 162/2023 de 02/03/2023" name="description">
   <link href="css/geral3.css" rel="stylesheet"/>
   <link href="https://d11gitgevq44cw.cloudfront.net/libs/font-awesome/5x/css/all.min.css" rel="stylesheet"/>
   <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" rel="stylesheet"/>
   <script crossorigin="anonymous" integrity="sha256-FgpCb/KJQlLNfOu91ta32o/NMZxltwRo8QtmkMRdAu8=" src="https://code.jquery.com/jquery-3.3.1.min.js">
   </script>
   <script crossorigin="anonymous" integrity="sha384-ApNb

#### Se olharmos atentamente este HTML, vamos perceber que os valores procurados se encontram principalmente nos elementos dt e dd.

In [3]:
# acessando o método findAll do BeatifulSoup
soup.findAll('dt')

[<dt class="col-sm-3">Reunião</dt>,
 <dt class="col-sm-3">Deliberação</dt>,
 <dt class="col-sm-3">Situação</dt>,
 <dt class="col-sm-3">Assunto</dt>,
 <dt class="col-sm-3">Autor</dt>]

In [4]:
# acessando o método findAll do BeatifulSoup
soup.findAll('dd')

[<dd class="col-sm-9">02/03/2023</dd>,
 <dd class="col-sm-9">02/03/2023</dd>,
 <dd class="col-sm-9">Proposição Despachada</dd>,
 <dd class="col-sm-9">Pontes, Bueiros, boca de lobo</dd>,
 <dd class="col-sm-9">Vereador <br/><b>Jessé Cléber de Souza</b>.</dd>]

Show! Mas também precisamos encontrar o texto de cada proposição. Para isso podemos acessar diretamente a classe onde o texto está sendo criado no HTML, ou seja, acessar a classe 'card-text'.

In [5]:
# encontrando o texto
soup.find(class_='card-text').get_text().strip()

'O vereador abaixo firmado requer seja encaminhada cópia da presente Indicação ao senhor Prefeito e à Secretaria de Obras solicitando a implantação de guarda-corpo na ponte de madeira situada na rua Artur Zarling, Warnow, trecho próximo à Serralheria Testoni.'

In [6]:
# colocando os valores em uma tabela do Pandas
cabecalhos = [e.get_text() for e in soup.findAll('dt')] + ['Texto'] # cria uma lista com o texto de cada uma das dt
df = pd.DataFrame({}, columns = cabecalhos) # criar um dataframe Vazio cujo cabeçalho é a lista obtida acima
df # visualizando o DataFrame

,Reunião,Deliberação,Situação,Assunto,Autor,Texto


In [7]:
# inserindo os valores em um DataFrame do Pandas
dt = soup.findAll('dt')
dd = soup.findAll('dd')
dic = {}


# utilizando um for para iterar entre os valores obtidos de dt e dd e os colocando em um dicionário
for i in range(len(dt)):
    x = dt[i].get_text()
    y = dd[i].get_text()
    dic[x] = y
    
# o texto não se encontra na
dic['Texto'] = soup.find(class_='card-text').get_text().strip()
dic ={k:[v] for k,v in dic.items()}
dic = pd.DataFrame(dic, columns  = cabecalhos)
df = pd.concat([df,dic], ignore_index=True)
df

,Reunião,Deliberação,Situação,Assunto,Autor,Texto
0,02/03/2023,02/03/2023,Proposição Despachada,"Pontes, Bueiros, boca de lobo",Vereador Jessé Cléber de Souza.,O vereador abaixo firmado requer seja encaminh...


### Tornando o webscrapping mais automatizado

Pronto, vimos como funciona o webscrapping para apenas uma página. Mas é necessário elaborar uma técnica que permita fazer esta extração de forma mais automatizada. 

Perceba que na url de cada proposição contém a proposição e o ano. Podemos tentar alterar isso via código! Automatizando nosso processo.

![](imagens/proposicao_numero.png)

In [8]:
# criando um dataframe vazio com os cabeçalhos
df = pd.DataFrame({}, columns = cabecalhos)
df

,Reunião,Deliberação,Situação,Assunto,Autor,Texto


In [9]:
# escolhendo uma proposição aleatória
proposicao = 1
ano = 2002

url = (f"https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1"
             "&nrProposicao=" + str(proposicao) + # inserindo a proposição
             "&aaProposicao=" + str(ano)) # inserindo o ano da proposição

# obtendo o html da página
html = requests.get(url, headers = headers, timeout = 10).content
# passando o html para o BeautifulSoup
soup = bs4.BeautifulSoup(html, 'html.parser')
# encontrando os valores nos elementos dt e dd do HTML
dt = soup.findAll('dt')
dd = soup.findAll('dd')
dic = {}

# iterando entre os valores encontrados e os colando em um dicionário
for i in range(len(dt)):
    x = dt[i].get_text()
    y = dd[i].get_text()
    dic[x] = y

dic['Texto'] = soup.find(class_='card-text').get_text().strip()
dic ={k:[v] for k,v in dic.items()}
dic = pd.DataFrame(dic, columns  = cabecalhos)
df = pd.concat([df,dic])
df

,Reunião,Deliberação,Situação,Assunto,Autor,Texto
0,01/02/2002,01/02/2002,Proposição Aprovada,Diversos,Vereador Rubens Reinhold Ittner.,"Roçar as laterais da rua três Corações, bairro..."


### Criando um buscador

Agora é hora de juntarmos todo este código e deixar de uma forma mais **pythonica**.
Inicialmente vamos criar uma classe que vai conter todas estas funções que já foram analisadas. Assim, para verificar o HTML da página ou Inserir os valores no banco, bastará apenas chamar a função.

In [10]:
# bibliotecas utilizadas dentro desta classe
import pandas as pd # pandas para trabalhar com tabelas
import bs4 # bs4 para trabalhar com HTML
import requests # requests para acessar a página web
import time # marcar o tempo que demora pra executar alguma função

class BuscadorPreposicoesIndaial:
    
    def __init__(self):
        self._cabecalhos = ['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor','Proposição','Ano','Texto']
        self._user_agent = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.63")
        self._df = pd.DataFrame({}, columns = self._cabecalhos)
    
    def getHTML(self, url):
        # obtendo o html da página
        html = requests.get(url, headers = {'User-Agent': self._user_agent}, timeout = 10).content
        # passando o html para o BeautifulSoup
        soup = bs4.BeautifulSoup(html, 'html.parser')
        return soup
    
    def getValuesfromHTML(self,soup,prop,ano):
        # criando um dicionário com os valores
        dt = soup.findAll('dt')
        dd = soup.findAll('dd')
        dic = {}

        for i in range(len(dt)):
            x = dt[i].get_text()
            y = dd[i].get_text()
            dic[x] = y

        # colocando o ano e a proposição para facilitar as referências no BI
        dic['Texto'] = soup.find(class_='card-text').get_text().strip() # retorna o texto da proposição
        dic['Proposição'] = prop # adicionando a proposição no dicionário 
        dic['Ano'] = ano # adicionando o ano no dicionário
        
        dic ={k:[v] for k,v in dic.items()}
        
        # criando um DataFrame com os valores
        dic = pd.DataFrame(dic, columns  = self._cabecalhos) # é preciso ser DataFrame para usar com o concat
        return dic
    
    def InserirValoresdf(self,novos_valores):
        self._df = pd.concat([self._df,novos_valores])
    
    def Atualiza(self, data_inicial, data_final):
        
        tempo_inicial = time.time()
        
        for y in range(data_inicial, data_final + 1):
            
            erro = 0
            prop = 1
            
            while erro < 6:
                
                url = (f"https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1"
                         "&nrProposicao=" + str(prop) + # Número da proposição
                         "&aaProposicao=" + str(y)) # Ano da proposição
                
                html = self.getHTML(url)
                
                if html.find(class_='col-10'):
                    erro += 1
                    pass
                else:
                    dic = self.getValuesfromHTML(html,prop,y)
                    self.InserirValoresdf(dic)
                
                if prop % 400 == 0:
                    tempo_atual = time.time()
                    tempo_transcorrido = round((tempo_atual - tempo_inicial) / 60 , 1)
                    print('Ano: {}, Proposição: {}, Tempo: {} min'.format(y,prop, tempo_transcorrido))
                          
                prop += 1
                time.sleep(0.35)

In [11]:
# inicializando o buscador
buscador = BuscadorPreposicoesIndaial()

# realizando uma busca entre duas datas
buscador.Atualiza(data_inicial = 1996,data_final = 1996)

Ano: 1996, Proposição: 400, Tempo: 3.2 min


In [12]:
# visualizando o DataFrame
df = buscador._df
df.reset_index(drop=True, inplace = True)
df

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto
0,22/02/1996,22/02/1996,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,1,1996,construção de Escola de 1º grau no Bairro Nova...
1,22/02/1996,22/02/1996,Proposição Aprovada,Serviços e Obras,Vereador Henrique Fritz.,2,1996,construção de Escola nas imediações dos Loteam...
2,22/02/1996,22/02/1996,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Henrique Fritz.,3,1996,alargamento da Rua ID 90
3,22/02/1996,22/02/1996,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,4,1996,prolongamento da rede d'água na Rua Reinhold S...
4,22/02/1996,22/02/1996,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,Vereador Henrique Fritz.,5,1996,Prolongamento de rede d'água na Rua Lorenz até...
...,...,...,...,...,...,...,...,...
391,17/10/1996,17/10/1996,Proposição Aprovada,Diversos,Vereador Sílvio Gonçalves da Luz.,395,1996,Efetuar melhorias na rede elétrica do Beco Rau...
392,21/10/1996,21/10/1996,Proposição Aprovada,Diversos,Vereador Antônio Carlos Fink.,396,1996,"Alterar o artigo 1º da Lei nº 1.255/82, que in..."
393,29/10/1996,29/10/1996,Proposição Aprovada,Diversos,Vereador Henrique Fritz.,397,1996,Efetuar levantamento dos artesões estabelecido...
394,02/12/1996,02/12/1996,Proposição Aprovada,Iluminação Pública e Rede de Energia Elétrica,Vereador Remir José de Faveri.,398,1996,Colocação de um transformador na Rua Venezuela...


# 3. Inserindo os registros em um Banco de Dados local

Agora é o momento que vamos transferir os dados para um Banco de Dados local. A estratégia que vamos utilizar aqui é transferir os registros a cada ano, ou seja, a cada iteração de nosso <i>for</i>.

In [13]:
# importando a bilbioteca para acessar o banco
# a sqlalchemy é uma biblioteca muito utilizada em conjunto com o pandas!
import sqlalchemy

# devemos inserir as constantes de nossa conexão com o banco
dados_conexao = (
    "Driver={SQL Server};" # tipo do banco utilizado
    "Server=DESKTOP-RFSFG8V;" # nome do servidor, também poderia ser localhost
    "Database=Indaial;" # nome da base de dados em que vamos inserir
)

# criando a chave para fazer a conexão
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(dados_conexao))

# criando a conexão com o banco de dados
conn = engine.connect()

Após conectar com o Banco de Dados, vamos testar um query para visualizar a tabela Indaial do nosso Banco de Dados.

In [14]:
# criando um queries no formato string para inserir junto de nossa conexão
script = f'''
SELECT * FROM Proposicoes

'''

pd.read_sql_query(script, conn) # lendo uma script sql com o Pandas

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto


Era este o resultado esperado. Ainda não acrescentamos xxx

### Adicionando os dados de 1996 no banco de dados

falar que vai fazer um teste de exemplo

In [15]:
df = buscador._df
# mudando o nome das colunas para ficar de acordo com o que foi criado no banco
df.columns = ['DataReuniao','DataDeliberacao','Situacao','Assunto','Autor','Proposicao','Ano','Texto']

# inserindo o DataFrame no Banco de Dados
df.to_sql('Proposicoes', engine, if_exists='append', index=False)

-1

Perceba que existiu um retorno igual a -1. Isso significa que tudo ocorreu bem na transferência dos dados para o banco.

![](imagens/insert_2.png)


Boa, adicionamos alguns dados em nossa base. Agora é preciso limpar o banco para inserir integralmente todos os registros! Neste utilizaremos o **TRUNCATE TABLE** que elimina todas as linhas de uma tabela!

In [20]:
from sqlalchemy.sql import text as sa_text

# escrevendo uma script
script = f'''
TRUNCATE TABLE Proposicoes

'''
# passando a script para o conector
conn.execute(sa_text(script).execution_options(autocommit=True))
# perceba que utilizamos o autocommit = True, isso ocorre porque temos que forçar a limpeza.
# utilizamos este parâmetro apenas para o Truncate
conn.close()

criar uma nova classe para o buscador

In [53]:
import pandas as pd
import bs4
import requests
import time
import sqlalchemy
from sqlalchemy.sql import text as sa_text

class BuscadorPreposicoesIndaial:
    
    def __init__(self):
        self._cabecalhos = ['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor','Proposição','Ano','Texto']
        self._user_agent = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.63")
        self._df = pd.DataFrame({}, columns = self._cabecalhos)
        self._dados_conexao = dados_conexao = ("Driver={SQL Server};Server=DESKTOP-RFSFG8V;Database=Indaial;")
    
    def ConectaBanco(self):
        
        self.engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(self._dados_conexao))
        conn = self.engine.connect()
        return conn
    
    def DesconectaBanco(self,conn):
        print('O Banco de Dados foi desconectado')
        conn.close()

    def getHTML(self, url):
        # obtendo o html da página
        html = requests.get(url, headers = {'User-Agent': self._user_agent}, timeout = 30).content
        # passando o html para o BeautifulSoup
        soup = bs4.BeautifulSoup(html, 'html.parser')
        return soup
    
    def getValuesfromHTML(self,soup,prop,ano):
        # criando um dicionário com os valores
        dt = soup.findAll('dt')
        dd = soup.findAll('dd')
        dic = {}

        for i in range(len(dt)):
            x = dt[i].get_text()
            y = dd[i].get_text()
            dic[x] = y

        # colocando o ano e a proposição para facilitar as referências no BI
        dic['Texto'] = soup.find(class_='card-text').get_text().strip() # retorna o texto da proposição
        dic['Proposição'] = prop # adicionando a proposição no dicionário 
        dic['Ano'] = ano # adicionando o ano no dicionário
        
        dic ={k:[v] for k,v in dic.items()}
        
        # criando um DataFrame com os valores
        dic = pd.DataFrame(dic, columns  = self._cabecalhos) # é preciso ser DataFrame para usar com o concat
        return dic
    
    def InserirValoresdf(self,novos_valores):
        self._df = pd.concat([self._df,novos_valores])
    
    def InserirBancoDeDados(self,y):
        
        self._df.columns = ['DataReuniao','DataDeliberacao','Situacao','Assunto','Autor','Proposicao','Ano','Texto']
        
        resp = self._df.to_sql('Proposicoes', self.engine, if_exists='append', index=False)
        
        if resp == -1:
            print('-'*20)
            print('Proposições do ano {} foram inseridas'.format(y))
            print('-'*20)
        else:
            print('*-*'*20)
            print('Proposições do ano {} não foram inseridas'.format(y))
            print('*-*'*20)
    
    def Atualiza(self, data_inicial, data_final):
        
        tempo_inicial = time.time()
        # conectar com o banco
        conn = self.ConectaBanco()
        
        for y in range(data_inicial, data_final + 1):
            
            erro = 0
            prop = 1
            
            while erro < 6:
                
                url = (f"https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1"
                         "&nrProposicao=" + str(prop) + # Número da proposição
                         "&aaProposicao=" + str(y)) # Ano da proposição
                
                html = self.getHTML(url)
                
                if html.find(class_='col-10'):
                    erro += 1
                    pass
                else:
                    dic = self.getValuesfromHTML(html,prop,y)
                    self.InserirValoresdf(dic)
                
                if prop % 100 == 0:
                    tempo_atual = time.time()
                    tempo_transcorrido = round((tempo_atual - tempo_inicial) / 60 , 1)
                    print('Ano: {}, Proposição: {}, Tempo: {} min'.format(y,prop, tempo_transcorrido))
                          
                prop += 1
                time.sleep(0.35)
        
            # inserir no banco de dados a cada passagem de ano
            print('*'*50)
            print('Indo para o banco de dados')
            self.InserirBancoDeDados(y)
            print('*'*50)
            # limpa o dataframe
            self._df = pd.DataFrame({}, columns = self._cabecalhos)
        
        # desconectar do banco
        print('*'*50)
        self.DesconectaBanco(conn)

    def LimpaBancoDeDados(self,conn):
        script = f'''
                    TRUNCATE TABLE Proposicoes
                '''
        conn.execute(sa_text(script).execution_options(autocommit=True))
            
        print('-'*50)
        print('A tabela foi limpa. Mas a conexão ainda está ativa!')
        print('-'*50)

In [51]:
buscador = BuscadorPreposicoesIndaial()
buscador.Atualiza(data_inicial = 1996, data_final = 1997)

Ano: 1996, Proposição: 100, Tempo: 0.8 min
Ano: 1996, Proposição: 200, Tempo: 1.6 min
Ano: 1996, Proposição: 300, Tempo: 2.4 min
Ano: 1996, Proposição: 400, Tempo: 3.2 min
**************************************************
Indo para o banco de dados
--------------------
Proposições do ano 1996 foram inseridas
--------------------
Ano: 1997, Proposição: 100, Tempo: 4.0 min
Ano: 1997, Proposição: 200, Tempo: 4.8 min
Ano: 1997, Proposição: 300, Tempo: 5.6 min
Ano: 1997, Proposição: 400, Tempo: 6.4 min
Ano: 1997, Proposição: 500, Tempo: 7.2 min
Ano: 1997, Proposição: 600, Tempo: 8.1 min
Ano: 1997, Proposição: 700, Tempo: 8.9 min
**************************************************
Indo para o banco de dados
--------------------
Proposições do ano 1997 foram inseridas
--------------------
**************************************************
Desconectando do banco de dados


Falar em acessar o máximo e fazer uma atualização incremental

In [57]:
conn = buscador.ConectaBanco()

script = f'''
SELECT
	MAX(ANO) AS ano_maximo,
	MAX(Proposicao) AS proposicao_maxima
FROM Proposicoes
WHERE
	Ano = (SELECT MAX(ANO) FROM Proposicoes)

'''

maximo = pd.read_sql_query(script, conn) # lendo uma script sql com o Pandas
maximo

,ano_maximo,proposicao_maxima
0,1997,694


In [60]:
ano_maximo, proposicao_maxima = maximo['ano_maximo'], maximo['proposicao_maxima']
ano_maximo[0], proposicao_maxima[0]

(1997, 694)

texto falando para arrumar a função atualiza

In [102]:
import pandas as pd
import bs4
import requests
import time
import sqlalchemy
from sqlalchemy.sql import text as sa_text

class BuscadorPreposicoesIndaial:
    
    def __init__(self):
        self._cabecalhos = ['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor','Proposição','Ano','Texto']
        self._user_agent = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.63")
        self._df = pd.DataFrame({}, columns = self._cabecalhos)
        self._dados_conexao = dados_conexao = ("Driver={SQL Server};Server=DESKTOP-RFSFG8V;Database=Indaial;")
    
    def ConectaBanco(self):
        self.engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(self._dados_conexao))
        conn = self.engine.connect()
        print('O Banco de Dados foi conectado')
        return conn
    
    def DesconectaBanco(self,conn):
        print('O Banco de Dados foi desconectado')
        conn.close()

    def getHTML(self, url):
        # obtendo o html da página
        html = requests.get(url, headers = {'User-Agent': self._user_agent}, timeout = 60).content
        # passando o html para o BeautifulSoup
        soup = bs4.BeautifulSoup(html, 'html.parser')
        return soup
    
    def getValuesfromHTML(self,soup,prop,ano):
        # criando um dicionário com os valores
        dt = soup.findAll('dt')
        dd = soup.findAll('dd')
        dic = {}

        for i in range(len(dt)):
            x = dt[i].get_text()
            y = dd[i].get_text()
            dic[x] = y

        # colocando o ano e a proposição para facilitar as referências no BI
        try:
            # https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=221&aaProposicao=2001
            # não tem texto
            dic['Texto'] = soup.find(class_='card-text').get_text().strip() # retorna o texto da proposição
        except:
            dic['Texto'] = ""
        dic['Proposição'] = prop # adicionando a proposição no dicionário 
        dic['Ano'] = ano # adicionando o ano no dicionário
        
        dic ={k:[v] for k,v in dic.items()}
        
        # criando um DataFrame com os valores
        dic = pd.DataFrame(dic, columns  = self._cabecalhos) # é preciso ser DataFrame para usar com o concat
        return dic
    
    def InserirValoresdf(self,novos_valores):
        self._df = pd.concat([self._df,novos_valores])
    
    def InserirBancoDeDados(self,y):
        
        self._df.columns = ['DataReuniao','DataDeliberacao','Situacao','Assunto','Autor','Proposicao','Ano','Texto']
        
        resp = self._df.to_sql('Proposicoes', self.engine, if_exists='append', index=False)
        
        if resp == -1:
            print('-'*20)
            print('Proposições do ano {} foram inseridas'.format(y))
            print('-'*20)
        else:
            print('*-*'*20)
            print('Proposições do ano {} não foram inseridas'.format(y))
            print('*-*'*20)
    
    def Atualiza(self,data_inicial, data_final,proposicao = 0):
        
        tempo_inicial = time.time()
        # conectar com o banco
        conn = self.ConectaBanco()
        
        for y in range(data_inicial, data_final + 1):
            
            erro = 0
            if y == data_inicial:
                prop = proposicao + 1
            else:
                prop = 1
            
            while erro < 6:
                
                url = (f"https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1"
                         "&nrProposicao=" + str(prop) + # Número da proposição
                         "&aaProposicao=" + str(y)) # Ano da proposição
                
                html = self.getHTML(url)
                
                if html.find(class_='col-10'):
                    erro += 1
                    pass
                else:
                    dic = self.getValuesfromHTML(html,prop,y)
                    self.InserirValoresdf(dic)
                
                if prop % 100 == 0:
                    tempo_atual = time.time()
                    tempo_transcorrido = round((tempo_atual - tempo_inicial) / 60 , 1)
                    print('Ano: {}, Proposição: {}, Tempo: {} min'.format(y,prop, tempo_transcorrido))
                          
                prop += 1
                time.sleep(0.35)
        
            # inserir no banco de dados a cada passagem de ano
            print('*'*50)
            self.InserirBancoDeDados(y)
            print('*'*50)
            # limpa o dataframe
            self._df = pd.DataFrame({}, columns = self._cabecalhos)
        
        
        del(tempo_inicial)
        # desconectar do banco
        self.DesconectaBanco(conn)

    
    def get_MaxValues(self,conn):
        script = f'''
        SELECT
            MAX(ANO) AS ano_maximo,
            MAX(Proposicao) AS proposicao_maxima
        FROM Proposicoes
        WHERE
            Ano = (SELECT MAX(ANO) FROM Proposicoes)

        '''

        maximo = pd.read_sql_query(script, conn)
        ano_maximo, proposicao_maxima = maximo['ano_maximo'], maximo['proposicao_maxima']
        return ano_maximo[0], proposicao_maxima[0]
    
    def LimpaBancoDeDados(self,conn):
        script = f'''
                    TRUNCATE TABLE Proposicoes
                '''
        conn.execute(sa_text(script).execution_options(autocommit=True))
            
        print('-'*50)
        print('A tabela foi limpa. Mas a conexão ainda está ativa!')
        print('-'*50)

In [75]:
buscador = BuscadorPreposicoesIndaial()
conn = buscador.ConectaBanco()
ano_maximo, proposicao_maximo = buscador.get_MaxValues(conn)
print(ano_maximo,proposicao_maximo)
buscador.DesconectaBanco(conn)

O Banco de Dados foi conecatdo
1997 694
O Banco de Dados foi desconectado


In [76]:
buscador.Atualiza(data_inicial = ano_maximo, data_final = ano_maximo+2,proposicao = proposicao_maximo)

O Banco de Dados foi conecatdo
Ano: 1997, Proposição: 700, Tempo: 0.0 min
**************************************************
Indo para o banco de dados
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
Proposições do ano 1997 não foram inseridas
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
**************************************************
Ano: 1998, Proposição: 100, Tempo: 0.9 min
Ano: 1998, Proposição: 200, Tempo: 1.7 min
Ano: 1998, Proposição: 300, Tempo: 2.5 min
Ano: 1998, Proposição: 400, Tempo: 3.3 min
Ano: 1998, Proposição: 500, Tempo: 4.2 min
Ano: 1998, Proposição: 600, Tempo: 5.0 min
Ano: 1998, Proposição: 700, Tempo: 5.9 min
Ano: 1998, Proposição: 800, Tempo: 6.7 min
**************************************************
Indo para o banco de dados
--------------------
Proposições do ano 1998 foram inseridas
--------------------
**************************************************
Ano: 1999, Proposição: 100, Tempo: 8.3 min
Ano: 1999, Proposição: 200, Tempo

##### Aqui foi feito apenas para os anos de 1998 e de 1999. Os demais anos foram feitos separadamente e não foram colocados aqui no código para evitar redundância.

# 4. Construção de um dashboard no Power BI

Após inserir nossos dados no SQL Server temos de levá-lo ao Power BI. Conseguimos fazer isso através da guia obter dados e selecionando o SQL Server. Então, colocamos nossas queries e trazemos as tabelas do banco de dados para nosso BI.

![](imagens/banco_de_dados_sql.png)

A partir disso é necessário realizar algumas transformações, como retirar a palavra "Vereador" da coluna Autor e criar uma tabela de Fato para os autores, uma vez que é possível uma proposição ter mais de um autor. Para mais detalhes, consulte o Power BI do arquivo que se encontra neste [link](https://github.com/FerrazThales/Observatorio_Social_Indaial).


O resultado de nosso BI se encontra na imagem abaixo:

![](imagens/dashboard_indaial.png)


* Na parte superior temos os KPIs que ajudam a entender de forma rápida o contexto geral das análises. Eles foram integralmente criados na linguagem DAX.
* Já na parte central é possível ter uma análise mais detalhada da criação das proposições e das atividades dos vereadores sobre diferentes assuntos.
* Por fim, na parte inferior, analisamos o avanço temporal da criação de proposição e quais foram as palavras que mais surgiram no texto das proposições.

# 5. Conclusão

Com esta raspagem de dados automatizada a população ganha agilidade para verificar a atividade dos vereadores de sua cidade. Além disso, é possível verificar quais as principais pautas estão sendo colocadas pelos políticos eleitos e fazer uma comparação das proposições levantadas em anos anteriores.